All imports required

In [ ]:
import glob
import tensorflow as tf
import keras
from keras import layers
import numpy as np
from xml.etree import ElementTree as ET
from PIL import ImageDraw, Image
import matplotlib.pyplot as plt
from google.colab import drive
import cv2
import os
!pip install -q tfds-nightly
import tensorflow_datasets as tfds
!pip install keras-tuner --upgrade
import keras_tuner as kt
import gdown

Download required files

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1ZbBsljRwlPSVCBWa_nJRJNwoblIEKDQE/view?usp=sharing
!tar -xzf /content/CandB.tar.gz
!gdown --fuzzy https://drive.google.com/file/d/1y8djbAYlOxd2TWGxlWUjBRG1HQ4WaPUj/view?usp=sharing
!tar -xzf /content/datas.tar.gz
!gdown --fuzzy https://drive.google.com/file/d/1aY_CLPxOBpAFm9zTekepMoBJ5qJvRolv/view?usp=sharing
!tar -xzf /content/Validation.tar.gz
!gdown --fuzzy https://drive.google.com/file/d/1BU-QcU4Ew_8YhAbqBulo4RXLsrMDxzZR/view?usp=sharing
!tar -xzf /content/validatas.tar.gz

Load dataset and create input pipeline

In [ ]:
def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255., label

In [ ]:
import datas
ds_train = tfds.load('datas', split=tfds.Split.TRAIN, as_supervised=True, shuffle_files=True)

ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.shuffle(buffer_size=1000)
ds_train = ds_train.batch(32)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
import validatas
ds_validate = tfds.load('validatas', split=tfds.Split.TRAIN, as_supervised=True, shuffle_files=True)

ds_validate = ds_validate.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_validate = ds_validate.shuffle(buffer_size=1000)
ds_validate = ds_validate.batch(32)
ds_validate = ds_validate.prefetch(tf.data.AUTOTUNE)

Create model

In [ ]:
model = keras.Sequential()
model.add(layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu", input_shape=(270,480,3)))
model.add(layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
model.add(layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(filters=48, kernel_size=3, padding="same", activation="relu"))
model.add(layers.Conv2D(filters=48, kernel_size=3, padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(layers.Conv2D(filters=64, kernel_size=3, padding="same", activation="relu"))
model.add(layers.MaxPooling2D(pool_size=2))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(12, activation="relu"))
model.add(layers.Dense(12, activation="relu"))
model.add(layers.Dense(4))

model.compile(optimizer=tf.optimizers.Adam(learning_rate=1e-3), 
              loss=tf.keras.losses.MeanAbsoluteError(), 
              metrics=[tf.keras.metrics.MeanAbsoluteError()])

Show model properties

In [ ]:
model.summary()

Train model

In [ ]:
history = model.fit(ds_train, epochs=20, validation_data=ds_validate)

Save and load model.
If you want to use it later you shoud save and download, or connect your google drive and change save path to smth like: "/content/drive/MyDrive/[...]",
how to connect drive: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA

In [ ]:
!mkdir saved_model
model.save("/content/saved_model")

In [ ]:
model = keras.models.load_model("/content/saved_model")

Showing loss function chart

In [ ]:
print(history.history.keys())
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

Show prediction (ds_validate can be changed to ds_train depending which dataset is to be predicted on)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

imgs = ds_validate.as_numpy_iterator().next()
pred = model.predict(imgs[0])
idx = 0

def convert(xy2):
    rect_y = xy2[0]*480
    rect_x = xy2[1]*480
    rect_yy = xy2[2]*480
    rect_xx = xy2[3]*480
    return rect_x, rect_y, rect_xx-rect_x, rect_yy-rect_y

_, ax = plt.subplots()
ax.imshow(imgs[0][idx])

x, y, w, h = convert(imgs[1][idx])
rect_true = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='b', facecolor='none')
ax.add_patch(rect_true)

x, y, w, h = convert(pred[idx])
rect_pred = patches.Rectangle((x, y), w, h, linewidth=2, edgecolor='r', facecolor='none')
ax.add_patch(rect_pred)

plt.show()